In [2]:
import yfinance as yf
import pandas as pd

# Fetch daily S&P 500 data
sp500_data = yf.download('^GSPC', start='2000-01-01', end='2024-01-01')

# Calculate additional features for S&P 500 data
sp500_data['daily_return'] = sp500_data['Close'].pct_change() #might need
sp500_data['volatility'] = sp500_data['daily_return'].rolling(window=20).std()

###sp500_data['ma_200'] = sp500_data['Close'].rolling(window=200).mean() # for long term


# Calculate the 200-day moving return
sp500_data['200_day_return'] = sp500_data['Close'].pct_change(periods=200)

[*********************100%%**********************]  1 of 1 completed


In [11]:
sp500 = sp500_data.dropna()
sp500.head()

,Open,High,Low,Close,Adj Close,Volume,daily_return,volatility,200_day_return
Date,,,,,,,,,
2000-10-17,1374.619995,1380.989990,1342.339966,1349.969971,1349.969971,1161500000,-0.017932,0.013576,-0.072326
2000-10-18,1349.969971,1356.650024,1305.790039,1342.130005,1342.130005,1441700000,-0.005808,0.013576,-0.040938
2000-10-19,1342.130005,1389.930054,1342.130005,1388.760010,1388.760010,1297900000,0.034743,0.016088,-0.009521
2000-10-20,1388.760010,1408.469971,1382.189941,1396.930054,1396.930054,1177400000,0.005883,0.016181,-0.004646
2000-10-23,1396.930054,1406.959961,1387.750000,1395.780029,1395.780029,1046800000,-0.000823,0.016139,-0.031697


In [4]:
import pandas_datareader.data as pdr
import yfinance as yf
import pandas as pd

# Fetch economic indicators from FRED
gdp = pdr.get_data_fred('GDP', start='2000-01-01', end='2024-01-01') #quaterly

inflation = pdr.get_data_fred('CPIAUCSL', start='2000-01-01', end='2024-01-01')  # CPI for inflation
unemployment = pdr.get_data_fred('UNRATE', start='2000-01-01', end='2024-01-01')
interest_rate = pdr.get_data_fred('FEDFUNDS', start='2000-01-01', end='2024-01-01')

# Rename columns for clarity
gdp.rename(columns={'GDP': 'GDP'}, inplace=True)
inflation.rename(columns={'CPIAUCSL': 'Inflation'}, inplace=True)
unemployment.rename(columns={'UNRATE': 'Unemployment'}, inplace=True)
interest_rate.rename(columns={'FEDFUNDS': 'Interest_Rate'}, inplace=True)

In [5]:
GDP = gdp.resample('D').ffill()
GDP.loc['2020-03-28':'2020-04-01'] 

,GDP
DATE,
2020-03-28,21706.513
2020-03-29,21706.513
2020-03-30,21706.513
2020-03-31,21706.513
2020-04-01,19913.143


In [7]:
Inflation = inflation.resample('D').ffill()
Unemployment = unemployment.resample('D').ffill()
Interest_rate = interest_rate.resample('D').ffill()

In [40]:
merged_data = sp500.join([Inflation, Unemployment, Interest_rate], how='left')



merged_data.head()

,Open,High,Low,Close,Adj Close,Volume,daily_return,volatility,200_day_return,Inflation,Unemployment,Interest_Rate
Date,,,,,,,,,,,,
2004-08-19,1095.170044,1095.170044,1086.280029,1091.229980,1091.229980,1.249400e+09,-0.003598,0.008767,0.038564,189.2,5.4,1.43
2004-08-20,1091.229980,1100.260010,1089.569946,1098.349976,1098.349976,1.199900e+09,0.006525,0.008592,0.037138,189.2,5.4,1.43
2004-08-23,1098.349976,1101.400024,1094.729980,1095.680054,1095.680054,1.021900e+09,-0.002431,0.008600,0.040285,189.2,5.4,1.43
2004-08-24,1095.680054,1100.939941,1092.819946,1096.189941,1096.189941,1.092500e+09,0.000465,0.008314,0.042194,189.2,5.4,1.43
2004-08-25,1096.189941,1106.290039,1093.239990,1104.959961,1104.959961,1.192200e+09,0.008000,0.008500,0.044336,189.2,5.4,1.43


Method 1

We use top 5 companies in US.


Apple, Microsoft, Amazon, Google, NVIDIA

Method 2

If we include some independent company as our feature, we avoid adding bias but also we are adding a feature that demonstrates the market state.

KKR,
Dell Technologies,	
Veeva Systems	

METHOD 1 :final_data





In [22]:
#notice the Close range for differ stocks 
# google has data only from 2004-08-19

tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'NVDA']

# Dictionary to store close price dataframes
close_prices = {}

# Fetch historical data for each ticker and extract the close prices
for ticker in tickers:
    stock = yf.Ticker(ticker)
    data = stock.history(start='2004-08-19', end='2024-01-01')
    close_prices[ticker] = data[['Close']].rename(columns={'Close': ticker})

# Combine all close price dataframes into a single dataframe
combined_close_prices = pd.concat(close_prices.values(), axis=1)
combined_close_prices.head()



,AAPL,MSFT,AMZN,GOOGL,NVDA
Date,,,,,
2004-08-19 00:00:00-04:00,0.463640,16.900316,1.9315,2.508132,0.089124
2004-08-20 00:00:00-04:00,0.464999,16.950171,1.9755,2.707353,0.094398
2004-08-23 00:00:00-04:00,0.469226,17.025167,1.9725,2.734599,0.096462
2004-08-24 00:00:00-04:00,0.482361,17.025167,1.9525,2.621365,0.092869
2004-08-25 00:00:00-04:00,0.498968,17.218922,2.0150,2.649612,0.096844


In [44]:
#change the date index of combined_close_prices to timezone-naive
combined_close_prices.index = combined_close_prices.index.tz_localize(None)
merged_data = merged_data.loc['2004-08-19':]

final_data = merged_data.join(combined_close_prices, how='inner')

final_data.head()


,Open,High,Low,Close,Adj Close,Volume,daily_return,volatility,200_day_return,Inflation,Unemployment,Interest_Rate,AAPL,MSFT,AMZN,GOOGL,NVDA
Date,,,,,,,,,,,,,,,,,
2004-08-19,1095.170044,1095.170044,1086.280029,1091.229980,1091.229980,1.249400e+09,-0.003598,0.008767,0.038564,189.2,5.4,1.43,0.463640,16.900316,1.9315,2.508132,0.089124
2004-08-20,1091.229980,1100.260010,1089.569946,1098.349976,1098.349976,1.199900e+09,0.006525,0.008592,0.037138,189.2,5.4,1.43,0.464999,16.950171,1.9755,2.707353,0.094398
2004-08-23,1098.349976,1101.400024,1094.729980,1095.680054,1095.680054,1.021900e+09,-0.002431,0.008600,0.040285,189.2,5.4,1.43,0.469226,17.025167,1.9725,2.734599,0.096462
2004-08-24,1095.680054,1100.939941,1092.819946,1096.189941,1096.189941,1.092500e+09,0.000465,0.008314,0.042194,189.2,5.4,1.43,0.482361,17.025167,1.9525,2.621365,0.092869
2004-08-25,1096.189941,1106.290039,1093.239990,1104.959961,1104.959961,1.192200e+09,0.008000,0.008500,0.044336,189.2,5.4,1.43,0.498968,17.218922,2.0150,2.649612,0.096844


METHOD 2 :final_data too 





In [75]:
#KKR has data only from 2010-07-15--should also change it ones before 

tickers = ['KKR', 'DELL', 'VEEV']
# Dictionary to store close price dataframes
close_prices = {}

# Fetch historical data for each ticker and extract the close prices
for ticker in tickers:
    stock = yf.Ticker(ticker)
    data = stock.history(start='2010-07-15', end='2024-01-01')
    close_prices[ticker] = data[['Close']].rename(columns={'Close': ticker})

combined_close_prices = combined_close_prices.ffill()

combined_close_prices.head()

,KKR,DELL,VEEV
Date,,,
2010-07-15,5.904121,11.303946,37.16
2010-07-16,5.608915,11.303946,37.16
2010-07-19,5.666799,11.303946,37.16
2010-07-20,5.354227,11.303946,37.16
2010-07-21,5.093751,11.303946,37.16


In [76]:
#change the date index of combined_close_prices to timezone-naive
combined_close_prices.index = combined_close_prices.index.tz_localize(None)
merged_data = merged_data.loc['2010-07-15':]

final_data = merged_data.join(combined_close_prices, how='inner')

final_data.head()

,Open,High,Low,Close,Adj Close,Volume,daily_return,volatility,200_day_return,Inflation,Unemployment,Interest_Rate,KKR,DELL,VEEV
Date,,,,,,,,,,,,,,,
2010-07-15,1094.459961,1098.660034,1080.530029,1096.479980,1096.479980,4.552470e+09,0.001196,0.012680,0.031515,217.605,9.4,0.18,5.904121,11.303946,37.16
2010-07-16,1093.849976,1093.849976,1063.319946,1064.880005,1064.880005,5.297350e+09,-0.028819,0.014131,0.004026,217.605,9.4,0.18,5.608915,11.303946,37.16
2010-07-19,1066.849976,1074.699951,1061.109985,1071.250000,1071.250000,4.089500e+09,0.005982,0.014231,0.013405,217.605,9.4,0.18,5.666799,11.303946,37.16
2010-07-20,1064.530029,1083.939941,1056.880005,1083.479980,1083.479980,4.713280e+09,0.011417,0.014533,0.052076,217.605,9.4,0.18,5.354227,11.303946,37.16
2010-07-21,1086.670044,1088.959961,1065.250000,1069.589966,1069.589966,4.747180e+09,-0.012820,0.014376,0.043289,217.605,9.4,0.18,5.093751,11.303946,37.16


In [77]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# y
y = np.where(final_data['Close'].shift(-1) > merged_data['Close'], 1, -1)#explain: if the next day's close price is higher than the current day's close price, the model should predict a 1, otherwise, it should predict a -1.
# feature
X = final_data.drop(columns=['Close', 'daily_return', '200_day_return']).shift(1)


y = y[:-1]
X = X.dropna()


y = y[-len(X):]

# Check the shape of 'y' and 'X'
print(y.shape, X.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Generate a classification report
report = classification_report(y_test, y_pred)
print(report)
# Calculate the accuracy score
accuracy = model.score(X_test, y_test)
print(f'Out-of-Sample Accuracy: {accuracy}')


###not improved much 

(3388,) (3388, 12)
              precision    recall  f1-score   support

          -1       0.47      0.45      0.46       308
           1       0.56      0.57      0.56       370

    accuracy                           0.52       678
   macro avg       0.51      0.51      0.51       678
weighted avg       0.52      0.52      0.52       678

Out-of-Sample Accuracy: 0.5176991150442478


In [78]:
# SVM model
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a StandardScaler() object
scaler = StandardScaler()


# Create an instance of SVM
model = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1, gamma=0.1))

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Generate a classification report
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

          -1       0.62      0.13      0.21       308
           1       0.56      0.93      0.70       370

    accuracy                           0.57       678
   macro avg       0.59      0.53      0.46       678
weighted avg       0.59      0.57      0.48       678



In [80]:
# linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# predict
y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, y_pred)
print(f'R2 Score: {r2}')
# accuracy


Mean Squared Error: 0.9934119325120795
R2 Score: -0.001789160198496731


In [81]:
# lstm with keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Normalize the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Create a Sequential model
model = Sequential()

# Add an LSTM layer
model.add(LSTM(100, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=10, shuffle=False)

# Make predictions
y_pred = model.predict(X_test_reshaped)

# Calculate the mean squared error

mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')

# accuracy
r2 = r2_score(y_test, y_pred)
print(f'R2 Score: {r2}')



/Users/qianqianmeng/.virtualenvs/r-reticulate/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.9977
Epoch 2/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9937
Epoch 3/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9932
Epoch 4/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9930
Epoch 5/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.9928
Epoch 6/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9926
Epoch 7/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9925
Epoch 8/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9924
Epoch 9/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9923
Epoch 10/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9922
Epoch 11/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9921
Epoch 12/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9921
Epoch 13/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9920
Epoch 14/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.9919
Epoch 15/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 

<!-- We create for sp500
Then we use same features for  -->